In [ ]:
import numpy as np
import pandas as pd
import snowflake.connector
from datetime import datetime, timedelta
import requests
import io

In [ ]:
#Importar planes de compra en formato csv
df_plan_fruver_nacional = pd.read_csv(r"C:\Users\juane\Downloads\PLAN_FRUVER_NACIONAL.csv")
df_plan_kvi_nacional = pd.read_csv(r"C:\Users\juane\Downloads\PLAN_KVI_NACIONAL.csv")
df_plan_fruver_bogota = pd.read_csv(r"C:\Users\juane\Downloads\PLAN_FRUVER_BOGOTA.csv")
df_plan_kvi_bogota = pd.read_csv(r"C:\Users\juane\Downloads\PLAN_KVI_BOGOTA.csv")

In [ ]:
#Definir variables con lista de skus a filtrar
sku_ids_to_filter = [3396,3404,3747,2530,2735]

In [ ]:
#Filtrar planes de compra respecto a la ciudad y los skus de trato especial
df_plan_fruver_nacional = df_plan_fruver_nacional[~df_plan_fruver_nacional['Sku Id'].isin(sku_ids_to_filter)]
df_plan_fruver_nacional = df_plan_fruver_nacional[df_plan_fruver_nacional['City ID'] != 'BogotC!Exito']
df_plan_kvi_nacional = df_plan_kvi_nacional[df_plan_kvi_nacional['Sku Id'].isin(sku_ids_to_filter)]
df_plan_kvi_nacional = df_plan_kvi_nacional[df_plan_kvi_nacional['City ID'] != 'BogotC!Exito']
df_plan_fruver_bogota = df_plan_fruver_bogota[~df_plan_fruver_bogota['Sku Id'].isin(sku_ids_to_filter)]
df_plan_fruver_bogota = df_plan_fruver_bogota[df_plan_fruver_bogota['City ID'] == 'BogotC!Exito']
df_plan_kvi_bogota = df_plan_kvi_bogota[df_plan_kvi_bogota['Sku Id'].isin(sku_ids_to_filter)]
df_plan_kvi_bogota = df_plan_kvi_bogota[df_plan_kvi_bogota['City ID'] == 'BogotC!Exito']

In [ ]:
#Cocatenar en uno los diferentes planes de compra
df_plan_fruver_colombia = pd.concat([df_plan_fruver_nacional, df_plan_kvi_nacional, df_plan_fruver_bogota, df_plan_kvi_bogota])
df_plan_fruver_colombia = df_plan_fruver_colombia.reset_index(drop=True)

In [ ]:
#Crear columna warehouse product id
df_plan_fruver_colombia['WAREHOUSE_PRODUCT_ID'] = df_plan_fruver_colombia['Warehouseid'].astype(str) + '-' + df_plan_fruver_colombia['Sku Id'].astype(str)

In [ ]:
# Lista de SKUs que deseas procesar
skus_a_procesar = [3396,3404,3747,2530,2735] 

# Filtrar el DataFrame para los SKUs específicos
df_plan_fruver_colombia.loc[df_plan_fruver_colombia['Sku Id'].isin(skus_a_procesar), 'ADJUSTED QUANTITY'] = (
    df_plan_fruver_colombia.loc[df_plan_fruver_colombia['Sku Id'].isin(skus_a_procesar)].apply(
        lambda row: round(max(row['Average Daily Sales for Past 7 Days']* 1, row['Purchase Quantity']), 0)
        if row['Purchase Quantity'] < row['Average Daily Sales for Past 7 Days'] * 1
        else round(row['Purchase Quantity'], 0), 
        axis=1
    )
)

# Lógica para los SKUs no especificados
df_plan_fruver_colombia.loc[~df_plan_fruver_colombia['Sku Id'].isin(skus_a_procesar), 'ADJUSTED QUANTITY'] = (
    df_plan_fruver_colombia.loc[~df_plan_fruver_colombia['Sku Id'].isin(skus_a_procesar)].apply(
        lambda row: round(max(row['Average Daily Sales for Past 7 Days'] * 0.4, row['Purchase Quantity']), 0)
        if row['Purchase Quantity'] < row['Average Daily Sales for Past 7 Days'] * 0.4
        else round(row['Purchase Quantity'], 0),
        axis=1
    )
)

In [ ]:
#Redondear las cantidades menores a 3
#df_plan_fruver_colombia['ADJUSTED QUANTITY'] = df_plan_fruver_colombia['ADJUSTED QUANTITY'].apply(
#    lambda x: 3 if x in [0, 1, 2] else x
#)

In [ ]:
#Ajustar las cantidades de las tiendas especificadas y redondear
# warehouses_to_adjust_quanity = [23]
# products_to_adjust_quanity = [6721,4376,12942]
# df_plan_fruver_colombia.loc[df_plan_fruver_colombia['Warehouseid'].isin(warehouses_to_adjust_quanity) & df_plan_fruver_colombia['Sku Id'].isin(products_to_adjust_quanity), 'ADJUSTED QUANTITY'] *= 0
# df_plan_fruver_colombia['ADJUSTED QUANTITY'] = df_plan_fruver_colombia['ADJUSTED QUANTITY'].round(0)

In [ ]:
#Ajustar las cantidades de las tiendas especificadas y redondear
warehouses_to_adjust_quanity = [103,106,104,109,93,2,79,90,4,49,63,66]
products_to_adjust_quanity = [2530, 2735, 3388, 3396, 3404, 3554, 3747, 3965, 4155, 4402, 12769]
df_plan_fruver_colombia.loc[df_plan_fruver_colombia['Warehouseid'].isin(warehouses_to_adjust_quanity) & df_plan_fruver_colombia['Sku Id'].isin(products_to_adjust_quanity), 'ADJUSTED QUANTITY'] *= 1.3
df_plan_fruver_colombia['ADJUSTED QUANTITY'] = df_plan_fruver_colombia['ADJUSTED QUANTITY'].round(0)

In [ ]:
#Añadir Banafrut al plan de compra con el % especificado de las cantidades, en las tiendas especificadas #MEDELLIN
rows_to_duplicate = df_plan_fruver_colombia[(df_plan_fruver_colombia['Warehouseid'].isin([21, 29, 31, 48, 52, 68])) & (df_plan_fruver_colombia['Sku Id'].isin([3396]))]
duplicated_rows = rows_to_duplicate.copy()
duplicated_rows['Primary Supplier Id'] = 346
duplicated_rows['ADJUSTED QUANTITY'] = (duplicated_rows['ADJUSTED QUANTITY'] * 0.7).round(0)
df_plan_fruver_colombia.loc[(df_plan_fruver_colombia['Warehouseid'].isin([21, 29, 31, 48, 52, 68])) & (df_plan_fruver_colombia['Sku Id'].isin([3396])), 'ADJUSTED QUANTITY'] = (df_plan_fruver_colombia['ADJUSTED QUANTITY'] * 0.3).round(0)
df_plan_fruver_colombia = pd.concat([df_plan_fruver_colombia, duplicated_rows], ignore_index=True)
df_plan_fruver_colombia = df_plan_fruver_colombia.reset_index(drop=True)

In [ ]:
#Añadir Banafrut al plan de compra con el % especificado de las cantidade,s en las tiendas especificadas #BOGOTA
#rows_to_duplicate = df_plan_fruver_colombia[(df_plan_fruver_colombia['Warehouseid'].isin([8, 47, 51])) & (df_plan_fruver_colombia['Sku Id'].isin([3396]))]
#duplicated_rows = rows_to_duplicate.copy()
#duplicated_rows['Primary Supplier Id'] = 346
#duplicated_rows['ADJUSTED QUANTITY'] = (duplicated_rows['ADJUSTED QUANTITY'] * 0.4).round(0)
#df_plan_fruver_colombia.loc[(df_plan_fruver_colombia['Warehouseid'].isin([8, 47, 51])) & (df_plan_fruver_colombia['Sku Id'].isin([3396])), 'ADJUSTED QUANTITY'] = (df_plan_fruver_colombia['ADJUSTED QUANTITY'] * 0.6).round(0)
#df_plan_fruver_colombia = pd.concat([df_plan_fruver_colombia, duplicated_rows], ignore_index=True)
#df_plan_fruver_colombia = df_plan_fruver_colombia.reset_index(drop=True)

In [ ]:
#Filtrar productos sustitutos y otros
df_plan_fruver_colombia = df_plan_fruver_colombia[~df_plan_fruver_colombia['Sku Id'].isin([14941, 14942, 6620])]

In [ ]:
#test comprar Niza directo de F9
warehouses_to_test = [18]
df_plan_fruver_colombia.loc[df_plan_fruver_colombia['Warehouseid'].isin(warehouses_to_test),'ADJUSTED QUANTITY'] = df_plan_fruver_colombia['Purchase Quantity (Min Val Adj.) Editable']

In [ ]:
#Exportar el plan de compra modificado en formato csv
fecha_actual = datetime.now().strftime("%d-%m-%Y")
nombre_archivo = f"C:/Users/juane/Downloads/PLAN_FRUVER_COLOMBIA_{fecha_actual}.csv"
df_plan_fruver_colombia.to_csv(nombre_archivo, index=False)

In [ ]:
#Crear el formato de subida masiva de ordenes de compra para WMS 
df_wms_input_fruver = pd.DataFrame()
df_wms_input_fruver['WAREHOUSE_DESTINATION'] = df_plan_fruver_colombia['Warehouseid']
df_wms_input_fruver['SUPPLIER_ID'] = df_plan_fruver_colombia['Primary Supplier Id']
df_wms_input_fruver['RETAIL_ID'] = df_plan_fruver_colombia['Sku Id']
df_wms_input_fruver['QUANTITY'] = df_plan_fruver_colombia['ADJUSTED QUANTITY'].astype(int)
df_wms_input_fruver['PLANNED_DATE'] = pd.to_datetime(datetime.now().date()) + pd.to_timedelta(df_plan_fruver_colombia['Supplier Lead Time (Days)'], unit='d')
df_wms_input_fruver['PLANNED_DATE'] = df_wms_input_fruver['PLANNED_DATE'].dt.strftime('%d-%m-%Y')
df_wms_input_fruver = df_wms_input_fruver.loc[df_wms_input_fruver['QUANTITY'] != 0].copy()
df_wms_input_fruver = df_wms_input_fruver.reset_index(drop=True)

In [ ]:
#Crear un formato de subida masiva exclusivo para los viernes, ya que se crean ordenes para sabado y domigo
if datetime.now().weekday() == 4:
    print("Hoy es viernes!")
    df_wms_input_fruver_domingo = df_wms_input_fruver[
        (df_wms_input_fruver['WAREHOUSE_DESTINATION'].isin([8, 9, 11, 12, 14, 16, 17, 18, 19, 23, 24, 28, 33, 47, 50, 51, 58, 60, 61, 67, 70, 72, 73, 78, 110])) & 
        (df_wms_input_fruver['RETAIL_ID'].isin([2530, 2735, 3388, 3396, 3404, 3554, 3747, 3965, 4155, 4402, 12769,4603,8599,6798]))
    ].copy()
    df_wms_input_fruver_domingo.loc[:, 'QUANTITY'] = (df_wms_input_fruver_domingo['QUANTITY'] * 0.6).round(0)
    df_wms_input_fruver_domingo.loc[:, 'PLANNED_DATE'] = pd.to_datetime(datetime.now().date()) + pd.to_timedelta(2, unit='d')
    df_wms_input_fruver_domingo['PLANNED_DATE'] = pd.to_datetime(df_wms_input_fruver_domingo['PLANNED_DATE'])
    df_wms_input_fruver_domingo['PLANNED_DATE'] = df_wms_input_fruver_domingo['PLANNED_DATE'].dt.strftime('%d-%m-%Y')
                                    
    df_wms_input_fruver.loc[(df_wms_input_fruver['WAREHOUSE_DESTINATION'].isin([8,9,11,12,14,16,17,18,19,23,24,28,33,47,50,51,58,60,61,67,70,72,73,78,110])) & 
                            (df_wms_input_fruver['RETAIL_ID'].isin([2530, 2735, 3388, 3396, 3404, 3554, 3747, 3965, 4155, 4402, 12769])), 'QUANTITY'] = (df_wms_input_fruver['QUANTITY'] * 0.65).round(0)
                                    
    nombre_archivo = f"C:/Users/juane/Downloads/WMS_INPUT_FRUVER_DOMINGO_{fecha_actual}.csv"
    df_wms_input_fruver_domingo.to_csv(nombre_archivo,index=False)
    
    for supplier_id in df_wms_input_fruver_domingo['SUPPLIER_ID'].unique():
        supplier_df = df_wms_input_fruver_domingo[df_wms_input_fruver_domingo['SUPPLIER_ID'] == supplier_id]
                                    
        file_name = f"C:/Users/juane/Downloads/WMS_INPUT_DOMINGO_FRUVER_supplier_{supplier_id}.csv"
        supplier_df.to_csv(file_name, index=False)                            
                                              

In [ ]:
#Exportar el formato de subida masiva en csv
nombre_archivo = f"C:/Users/juane/Downloads/WMS_INPUT_FRUVER_{fecha_actual}.csv"
df_wms_input_fruver.to_csv(nombre_archivo,index=False)

In [ ]:
#Crear un formato de subida masiva de ordenes para las tiendas en Bogotá con los productos especificados unicamente
filtered_df = df_wms_input_fruver[
   (df_wms_input_fruver['WAREHOUSE_DESTINATION'].isin([8, 9, 11, 12, 14, 16, 17, 18, 19, 23, 24, 28, 33, 47, 50, 51, 58, 60, 61, 67, 70, 72, 73, 78, 110])) &
    (df_wms_input_fruver['RETAIL_ID'].isin([2530, 2735, 3396, 3404, 3747]))
]

for supplier_id in filtered_df['SUPPLIER_ID'].unique():
    supplier_df = filtered_df[filtered_df['SUPPLIER_ID'] == supplier_id]
    
    file_name = f"C:/Users/juane/Downloads/WMS_INPUT_FRUVER_bogota_productos_top_{supplier_id}.csv"
    supplier_df.to_csv(file_name, index=False)

In [ ]:
#Crear un formato de subida masiva de ordenes para todas las tiendas y productos restantes
filtered_out_df = df_wms_input_fruver[
    (df_wms_input_fruver['WAREHOUSE_DESTINATION'].isin([8, 9, 11, 12, 14, 16, 17, 18, 19, 20, 23, 24, 26, 28, 33, 47, 50, 51, 58, 60, 61, 67, 70, 72, 73, 78, 110])) &
    (df_wms_input_fruver['RETAIL_ID'].isin([2530, 2735, 3396, 3404, 3747]))
]

remaining_df = df_wms_input_fruver[~df_wms_input_fruver.index.isin(filtered_out_df.index)]

for supplier_id in remaining_df['SUPPLIER_ID'].unique():
    supplier_df = remaining_df[remaining_df['SUPPLIER_ID'] == supplier_id]
    
    file_name = f"C:/Users/juane/Downloads/WMS_INPUT_FRUVER_supplier_{supplier_id}.csv"
    supplier_df.to_csv(file_name, index=False)